In [ ]:
# Instalación (Colab/Local)
# Instalación (Colab/Local)
!pip install -U "langchain>=0.2.11" "langchain-core>=0.2.33" "langchain-community>=0.2.11" "langchain-openai>=0.2.1" "langsmith>=0.1.97"
# Opcionales para el assignment:
!pip install -U faiss-cpu chromadb tavily-python duckduckgo-search langchain-text-splitters

import os
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "api-key"
os.environ["OPENAI_API_KEY"] = "api-key"

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-5-mini", temperature=0)  # modelo sugerido

# Hola LLM
resp = llm.invoke("Definí 'Transformer' en una sola oración.")
print(resp.content)

Un Transformer es una arquitectura de red neuronal que usa mecanismos de atención (self-attention) para modelar dependencias de largo alcance en secuencias y procesarlas en paralelo para tareas como traducción y comprensión de lenguaje.


In [6]:
from langchain_openai import ChatOpenAI

# Completar parámetros básicos (ver [1] y [12])
MODEL = "gpt-5-mini"
TEMP = 0

llm = ChatOpenAI(model=MODEL, temperature=TEMP)
print(llm.invoke("Hola! Decime tu versión en una línea.").content)

Mi versión: ChatGPT — modelo GPT-4o de OpenAI.


In [7]:
from langchain_openai import ChatOpenAI

MODEL = "gpt-5-mini"
TEMP = 0

llm = ChatOpenAI(model=MODEL, temperature=TEMP)
print(llm.invoke("Escribí un haiku sobre evaluación de modelos.").content)

Mide la verdad
curva, ruido y sesgo
aprende siempre


In [8]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Sos un asistente conciso, exacto y profesional."),
    ("human",  "Explicá {tema} en <= 3 oraciones, con un ejemplo real.")
])

chain = prompt | llm  # LCEL: prompt → LLM
print(chain.invoke({"tema": "atención multi-cabeza"}).content)

La atención multi-cabeza proyecta consultas, claves y valores en varias subespacios (cabezas), calcula en cada uno la atención por producto punto escalada y concatena esas salidas para una proyección final.  
Así el modelo puede atender simultáneamente a distintos tipos de relaciones (p. ej. sintácticas vs. semánticas) y a distintos rangos posicionales.  
Ejemplo: en traducción, una cabeza puede alinear "El gato"→"The cat" mientras otra identifica la relación verbo‑objeto para ordenar y elegir la forma verbal correcta al generar "eats the fish".


In [9]:
from typing import List
from pydantic import BaseModel

class Resumen(BaseModel):
    title: str
    bullets: List[str]

llm_json = llm.with_structured_output(Resumen)  # garantiza JSON válido que cumple el esquema

pedido = "Resumí en 3 bullets los riesgos de la 'prompt injection' en LLM apps."
res = llm_json.invoke(pedido)
res

Resumen(title='Riesgos de la prompt injection en aplicaciones con LLM', bullets=['Ejecución de instrucciones maliciosas: el atacante puede inducir al modelo a realizar acciones no autorizadas (exfiltrar datos, ejecutar comandos, acceder a recursos) comprometiendo seguridad operacional.', 'Manipulación del comportamiento del modelo: prompts maliciosos pueden inducir respuestas incorrectas, sesgadas o que evadan filtros y mecanismos de seguridad, generando desinformación y fallas de control.', 'Pérdida de privacidad y confianza: exposición de datos sensibles, filtrado de información confidencial y daño reputacional o legal para la organización que despliega la aplicación.'])

In [10]:
_ = (prompt | llm).invoke({"tema": "transformers vs RNNs"})
print("Traza enviada (ver LangSmith).")

Traza enviada (ver LangSmith).


In [11]:
# Esqueleto sugerido para 1) y 2)
from pydantic import BaseModel

class Traduccion(BaseModel):
    text: str
    lang: str

traductor = llm.with_structured_output(Traduccion)
salida = traductor.invoke("Traducí al portugués: 'Excelente trabajo del equipo'.")
print(salida)

# Q&A con contexto (sin RAG)
from langchain_core.prompts import ChatPromptTemplate
QA_prompt = ChatPromptTemplate.from_messages([
    ("system", "Respondé SOLO usando el contexto. Si no alcanza, decí 'No suficiente contexto'."),
    ("human",  "Contexto:\n{contexto}\n\nPregunta: {pregunta}\nRespuesta breve:")
])
salida = (QA_prompt | llm).invoke({
    "contexto": "OpenAI y LangChain permiten structured output con JSON...",
    "pregunta": "¿Qué ventaja tiene structured output?"
})
print(salida)

text='Excelente trabalho da equipe' lang='pt'


content='Permite producir salida estructurada en formato JSON.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 211, 'prompt_tokens': 54, 'total_tokens': 265, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CartqNO8X1jArrSCzSBmB8dCYgHXY', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--87795517-802e-413f-a454-846edaed9920-0' usage_metadata={'input_tokens': 54, 'output_tokens': 211, 'total_tokens': 265, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 192}}


In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

# Zero-shot
zs_prompt = ChatPromptTemplate.from_messages([
    ("system", "Sos un asistente conciso y exacto."),
    ("human",  "Clasificá el sentimiento de este texto como POS, NEG o NEU:\n\n{texto}")
])

# Few-shot (1–2 ejemplos)
fs_prompt = ChatPromptTemplate.from_messages([
    ("system", "Sos un asistente conciso y exacto."),
    ("human",  "Ejemplo:\nTexto: 'El producto superó mis expectativas'\nEtiqueta: POS"),
    ("human",  "Ejemplo:\nTexto: 'La entrega fue tarde y vino roto'\nEtiqueta: NEG"),
    ("human",  "Texto: {texto}\nEtiqueta:")
])

textos = [
    "Me encantó la experiencia, repetiría.",
    "No cumple lo prometido; decepcionante.",
    "Está bien, nada extraordinario."
]

print("== Zero-shot ==")
for t in textos:
    print((zs_prompt | llm).invoke({"texto": t}).content)

print("\n== Few-shot ==")
for t in textos:
    print((fs_prompt | llm).invoke({"texto": t}).content)

== Zero-shot ==


POS


NEG


NEU

== Few-shot ==


POS


Etiqueta: NEG


NEU


In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


long_text = """A lo Vivo
Era un pueblecito rayano, Ribamoura, vivero de contrabandistas, donde esta profesión de riesgo y lucro hacía a la gente menos dormida de lo que suelen ser los pueblerinos. Abundaban los mozos de cabeza caliente, y se desdeñaba al que no era capaz de coger una escopeta y salir a la ganancia.

Las mujeres, vestidas y adornadas con lo que da de sí el contrabando, lucían pendientes de ostentosa filigrana, patenas fastuosas, pañuelos de seda de colorines; en las casas no faltaba ron jamaiqueño ni queso de Flandes, y los hombres poseían armas inglesas, bolsas de piel y tabaco Virginia y Macuba. Al través de Portugal, Inglaterra enviaba sus productos, y de España pasaban otros, cruzando el caudaloso río.

Algunos días del año se interrumpía el tráfico y la industria de Ribamoura. El pueblo entero se congregaba a celebrar las solemnidades consuetudinarias, que servían de pretexto para solaces y holgorio. Tal ocurría con el Carnaval, tal con la fiesta de la Patrona, tal con los días de la Semana Santa. A pesar de ser éstos de penitencia y mortificación, para los de Ribamoura tenían carácter de fiesta; en ellos se celebraba, en la iglesia principal, espacioso edificio de la época herreriana, la representación de la Pasión, con personajes de carne y hueso, y encargándose de los papeles gente del pueblo mismo.

Venido de Oporto, un actor portugués, con el instinto dramático de la raza, organizaba y dirigía la representación; pero sin tomar parte en ella. Esto se hubiese considerado en Ribamoura irreverente. «Trabajaban» por devoción y por respeto tradicional a los misterios redentores; pero nunca hubiesen admitido a nadie mercenario, ni tolerado que hiciese los papeles nadie de mala reputación. Gente honrada, aunque contrabandease; que eso no deshonra. Ni por pecado lo daban en el confesionario los frailes."""

# Split en chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=100)
chunks = splitter.split_text(long_text)

# Cadena para resumir un chunk
chunk_summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "Resumí el siguiente fragmento en 2–3 bullets, claros y factuales."),
    ("human", "{input}")
])

llm = ChatOpenAI(model="gpt-5-mini", temperature=0)
chunk_summary = chunk_summary_prompt | llm

bullets = [chunk_summary.invoke({"input": c}).content for c in chunks]

# Reduce (combinar resultados)
reduce_prompt = ChatPromptTemplate.from_messages([
    ("system", "Consolidá bullets redundantes y producí un resumen único y breve."),
    ("human", "Bullets:\n{bullets}\n\nResumen final (<=120 tokens):")
])

final = (reduce_prompt | llm).invoke({"bullets": "\n".join(bullets)}).content
print(final)

Ribamoura, pueblo fronterizo famoso por el contrabando, vivía de él: joyas y sedas, ron y quesos, armas y tabacos entraban por Portugal o cruzando el río. La actividad, arriesgada y lucrativa, marcaba la conducta social (se valoraba a quien salía armado). El pueblo se reunía en festejos tradicionales—Carnaval, la Patrona y una Semana Santa festiva con una Pasión representada por vecinos bajo la dirección de un actor de Oporto—siempre devota y excluyente de mercenarios. Aun contrabandistas, se consideraban gente honrada; los frailes no lo veían como pecado.


In [14]:
from typing import List, Optional
from pydantic import BaseModel
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

class Entidad(BaseModel):
    tipo: str   # p.ej., 'ORG', 'PER', 'LOC'
    valor: str

class ExtractInfo(BaseModel):
    titulo: Optional[str]
    fecha: Optional[str]
    entidades: List[Entidad]

extractor = llm.with_structured_output(ExtractInfo)
texto = "OpenAI anunció una colaboración con la Universidad Catolica del Uruguay en Montevideo el 05/11/2025."
extractor.invoke(f"Extraé titulo, fecha y entidades (ORG/PER/LOC) del siguiente texto:\n\n{texto}")

ExtractInfo(titulo=None, fecha='05/11/2025', entidades=[Entidad(tipo='ORG', valor='OpenAI'), Entidad(tipo='ORG', valor='Universidad Catolica del Uruguay'), Entidad(tipo='LOC', valor='Montevideo')])

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS  # Si usás Chroma, importá su VectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain
from langchain_core.documents import Document

docs_raw = [
    "LangChain soporta structured output con Pydantic.",
    "RAG combina recuperación + generación para mejor grounding.",
    "OpenAIEmbeddings facilita embeddings para indexar textos."
]
docs = [Document(page_content=t) for t in docs_raw]

# Split y vector store
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)

emb = OpenAIEmbeddings()
vs = FAISS.from_documents(chunks, embedding=emb)
retriever = vs.as_retriever(search_kwargs={"k": 4})

llm = ChatOpenAI(model="gpt-5-mini", temperature=0)
prompt = ChatPromptTemplate.from_messages([
    ("system", "Respondé SOLO con el contexto. Si no alcanza, decí 'No suficiente contexto'."),
    ("human",  "Contexto:\n{context}\n\nPregunta: {input}")
])

combine_docs_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

rag_chain.invoke({"input": "¿Qué ventaja clave aporta RAG?"})

{'input': '¿Qué ventaja clave aporta RAG?',
 'context': [Document(id='072d06e5-4c87-4cdf-b74c-04b8e5ce5b98', metadata={}, page_content='RAG combina recuperación + generación para mejor grounding.'),
  Document(id='a43dd93c-7eb4-4337-9091-bd377854c8d1', metadata={}, page_content='OpenAIEmbeddings facilita embeddings para indexar textos.'),
  Document(id='b581ab9d-5f73-4f84-a3ec-2da4b000da88', metadata={}, page_content='LangChain soporta structured output con Pydantic.')],
 'answer': 'RAG combina recuperación + generación para mejor grounding.\n\nOpenAIEmbeddings facilita embeddings para indexar textos.\n\nLangChain soporta structured output con Pydantic.'}